In [77]:
import dill
import json
from string import punctuation
from re import search
import os

In [80]:
BASELINE_TRAJECTORY_PATH = 'trajectories_gemini'
N_FILES = 60

In [81]:
envs_bagel = {'click-checkboxes-soft':10,
 'click-tab-2-hard':20,
 'social-media':15,
 'email-inbox':30,
 'social-media-some':30,
 'tic-tac-toe':10,
 'use-autocomplete':10,
 'book-flight':30,
 'choose-date':20,
 'search-engine':20}

In [96]:
scoring_function = {
    'click-checkboxes-soft': eval_click_checkboxes_soft,
    'click_tab_2_hard': eval_click_tab_2_hard,
    'social-media': eval_social_media
}


In [98]:
def find_ref(target, trajectory):
    for state in trajectory['states']:
        for element in state['dom_elements']:
            ref = element['ref']
            text = element['text']
            if text == target:
                return ref
    return 0

def action_found(ref, trajectory):
    for state in trajectory['states']:
        if 'CLICK_ELEMENT' in state['action']:
            action_ref = int(search('CLICK_ELEMENT ([0-9]*)',state['action']).group(1))
            if action_ref ==  ref:
                return True
    return False

def eval_click_tab_2_hard(trajectory):
    target = search('\"(.*)\"', trajectory['utterance']).group(1)#.strip(punctuation).strip()
    ref = find_ref(target, trajectory)
    if action_found(ref,trajectory):
        return 1
    return 0

def eval_click_checkboxes_soft(trajectory):
    return 0

def eval_social_media(trajectory):
    return 0

def score_env(env_name):
    path = BASELINE_TRAJECTORY_PATH+'/'+env_name
    files = os.listdir(path)
    score = 0
    reward = 0
    for i in range(N_FILES):
        filepath = path + '/' + files[i]
        print(filepath)
        trajectory = dill.load(open(filepath, 'rb'))
        score += scoring_function[env_name](trajectory)
        traj_reward = trajectory['reward']
        if traj_reward is not None and traj_reward > 0:
            reward += 1
    return score/N_FILES, reward/N_FILES

    
score_env('click-checkboxes-soft')

trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft37.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft39.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft53.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft18.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft52.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft48.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft02.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft45.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft50.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft14.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft41.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft10.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft60.pkd
trajectories_gemini/click-checkboxes-soft/click-checkboxes-soft19.pkd
trajectories_gemini/

IsADirectoryError: [Errno 21] Is a directory: 'trajectories_gemini/click-checkboxes-soft/.ipynb_checkpoints'

In [87]:
env = list(envs_bagel.keys())[2]
env

'social-media'

In [92]:
path = BASELINE_TRAJECTORY_PATH + '/' + env + '/' + env + '02.pkd'
trajectory = dill.load(open(path,'rb'))
trajectory

{'utterance': 'For the user @fames, click on the "Like" button.',
 'reward': 0.7360666666666666,
 'states': [{'time': 0,
   'action': '',
   'dom_elements': ({'ref': 1,
     'parent': 0,
     'left': array([0.], dtype=float32),
     'top': array([0.], dtype=float32),
     'width': array([500.], dtype=float32),
     'height': array([210.], dtype=float32),
     'tag': 'body',
     'text': '',
     'value': '',
     'id': '',
     'classes': '',
     'bg_color': array([0.33333334, 0.33333334, 0.33333334, 1.        ], dtype=float32),
     'fg_color': array([0., 0., 0., 1.], dtype=float32),
     'flags': array([1, 0, 0, 0], dtype=int8)},
    {'ref': 2,
     'parent': 1,
     'left': array([0.], dtype=float32),
     'top': array([0.], dtype=float32),
     'width': array([160.], dtype=float32),
     'height': array([210.], dtype=float32),
     'tag': 'div',
     'text': '',
     'value': '',
     'id': 'wrap',
     'classes': '',
     'bg_color': array([1., 1., 1., 1.], dtype=float32),
     '

In [91]:
[x['action'] for x in trajectory['states']]

['',
 'Click on Embed Tweet button of the tweet by user @turpis. - CLICK_ELEMENT 42']

In [63]:
type(trajectory['states'][0]['dom_elements'])

tuple

In [41]:
path = MANUAL_TRAJECTORY_PATH + '/' + env + '.json'
path

'sample_trajectories/use-autocomplete.json'

In [42]:
with open(path,'r') as f:

    manual_trajectory = json.load(f)

In [43]:
[x['action'] for x in manual_trajectory['states']]

[None,
 {'type': 'mousedown', 'x': 50, 'y': 75, 'timing': 1},
 {'type': 'mousedown', 'x': 50, 'y': 75, 'timing': 3},
 {'type': 'mouseup', 'x': 50, 'y': 76, 'timing': 1},
 {'type': 'mouseup', 'x': 50, 'y': 76, 'timing': 3},
 {'type': 'click', 'x': 50, 'y': 76, 'timing': 1},
 {'type': 'click', 'x': 50, 'y': 76, 'timing': 3},
 {'type': 'keydown', 'keyCode': 16, 'charCode': 0, 'timing': 1},
 {'type': 'keydown', 'keyCode': 16, 'charCode': 0, 'timing': 3},
 {'type': 'keydown', 'keyCode': 84, 'charCode': 0, 'timing': 1},
 {'type': 'keydown', 'keyCode': 84, 'charCode': 0, 'timing': 3},
 {'type': 'keypress', 'keyCode': 84, 'charCode': 84, 'timing': 1},
 {'type': 'keypress', 'keyCode': 84, 'charCode': 84, 'timing': 3},
 {'type': 'keyup', 'keyCode': 16, 'charCode': 0, 'timing': 1},
 {'type': 'keyup', 'keyCode': 16, 'charCode': 0, 'timing': 3},
 {'type': 'keyup', 'keyCode': 84, 'charCode': 0, 'timing': 1},
 {'type': 'keyup', 'keyCode': 84, 'charCode': 0, 'timing': 3},
 {'type': 'keydown', 'keyCode